In [2]:
from typing import List
from functools import lru_cache

import arrays
from importlib import reload
reload(arrays)
import arrays as ar


In [92]:
def min_histogram_rectangle(h: list) -> int:
    s = []
    mx = [0]
    def measure(j, v):
        while s and h[s[-1]] > v:
            t = s.pop()
            i = s[-1] if s else -1
            mx[0] = max(mx[0], (j - i -1) * h[t])

    for j, v in enumerate(h):
        measure(j, v)
        s.append(j)
    measure(len(h), 0)
    return mx[0]


In [93]:
min_histogram_rectangle([6,2,5,4,5,1,6])


0 [] + [6] 
1 [6] + [2]  -> 6
2 [2] + [5] 
3 [2, 5] + [4]  -> 6
4 [2, 4] + [5] 
5 [2, 4, 5] + [1]  -> 6 -> 12 -> 12
6 [1] + [6] 
7 [1, 6] + [0]  -> 12 -> 12


12